In [1]:
import matplotlib.pyplot as plt      
from mpl_toolkits import mplot3d
import os
import sys
import glob
import time
import shutil
import gym
from gym import error, spaces, utils
import numpy as np
import math

In [2]:
import gym_fish

In [3]:
from gym_fish.envs.lib import pyflare as fl

In [4]:
Experiment_name = 'reach target with buoyancy control'
Experiment_note = """
"""

In [5]:
gpuId = 0
control_dt=0.2
theta = np.array([90,90])
phi = np.array([45,45])
radius=2.0
max_time = 10
action_max= 10
done_dist = 0.15
dist_distri_param =np.array([0,0.0])
wc = 0.0*np.array([1.0,0.5])
wp = 1.0*np.array([0.0,1.0])
wa = 0.5

couple_mode =  fl.COUPLE_MODE.TWO_WAY

In [6]:
env=gym.make('fish-basic-v0',
             env_json='../assets/env_file/env_basic.json',
                  gpuId=gpuId,
                        couple_mode=couple_mode,
                         control_dt=control_dt,
                         theta = theta,
             phi = phi,
                        radius=radius,
             action_max=action_max,
                        max_time=max_time,
                        wp=wp,wc=wc,wa=wa,
                       done_dist=done_dist,dist_distri_param=dist_distri_param
                   )
env_test=gym.make('fish-basic-v0',
             env_json='../assets/env_file/env_basic.json',
                  gpuId=gpuId,
                        couple_mode=couple_mode,
                         control_dt=control_dt,
                         theta = theta,
             phi = phi,
                        radius=radius,
             action_max=action_max,
                        max_time=max_time,
                        wp=wp,wc=wc,wa=wa,
                       done_dist=done_dist,dist_distri_param=dist_distri_param
                   )

visulization data save folder /home/liuwj/codes/pytorch_fish/scripts/gym-fish/gym_fish/envs/data/vis_data/


/home/liuwj/test_env/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


visulization data save folder /home/liuwj/codes/pytorch_fish/scripts/gym-fish/gym_fish/envs/data/vis_data/


In [7]:
print(os.getcwd())
os.chdir(os.getcwd()+"/../")
print(os.getcwd())

/home/liuwj/codes/pytorch_fish/scripts
/home/liuwj/codes/pytorch_fish


### Import Settings

In [8]:
cwd = os.getcwd()+"/"
print(cwd)

/home/liuwj/codes/pytorch_fish/


In [9]:
def plot_infos(infos,title,reward_fig_name):
    dd = {k:[] for k in infos[0].keys()}
    for info in infos:
        for k in info.keys():
            dd[k].append(info[k])
    plt.figure()
    for k in dd.keys():
        if k=='action_penality':
            plt.plot(np.arange(0,len(dd[k])),[kkk  for kkk in dd[k]],label=k)
        else:
            plt.plot(np.arange(0,len(dd[k])),dd[k],label=k)
    plt.legend()
    plt.title(title)
    plt.savefig(reward_fig_name)
    plt.show()

### Output Settings

In [10]:
store_folder_name = time.strftime('%Y-%m-%d/',time.localtime(time.time()))+Experiment_name+'_'+time.strftime('%Y-%m-%d %H:%M/',time.localtime(time.time()))
imgs_folder_name = 'imgs/'
network_folder_name = 'networks/'
store_folder = cwd+'output_data/'+store_folder_name
imgs_folder = store_folder+imgs_folder_name
network_folder = store_folder+network_folder_name

if not os.path.exists(store_folder):
    os.makedirs(store_folder)
if not os.path.exists(imgs_folder):
    os.makedirs(imgs_folder)
if not os.path.exists(network_folder):
    os.makedirs(network_folder)

### RL_TRAINING Setup

In [11]:
from stable_baselines3 import SAC

## RL Begin

In [12]:
algo = 'SAC'
algofoler=algo+'/'
if not os.path.exists(network_folder+algofoler):
    os.makedirs(network_folder+algofoler)
if not os.path.exists(network_folder+algofoler+'imgs/'):
    os.makedirs(network_folder+algofoler+'imgs/')
if not os.path.exists(network_folder+algofoler+'imgs/trajs/'):
    os.makedirs(network_folder+algofoler+'imgs/trajs/')
if not os.path.exists(network_folder+algofoler+'imgs/rewards/'):
    os.makedirs(network_folder+algofoler+'imgs/rewards/')
if not os.path.exists(network_folder+algofoler+'models/'):
    os.makedirs(network_folder+algofoler+'models/')

tb_folder =  network_folder+algofoler
model_folder = network_folder+algofoler+'models/'
result_img_folder = network_folder+algofoler+'imgs/'

In [13]:
def evaluate_env(env_test,model,title,traj_fig_name,reward_fig_name,save=False):
    obs = env_test.reset()
    dones = False
    infos = []
    reward = 0
    while not dones:
        action, _states = model.predict(obs, deterministic=True)
        if not save:
            obs, r, dones,info=env_test.step(action)
        else:
            obs, r, dones,info=env_test.stepSave(action,save_fluid=True)
        reward=reward+r
        infos.append(info)
    plot_infos(infos,title=title,reward_fig_name=reward_fig_name)
    env_test.plot3d(title=title,fig_name = traj_fig_name)

### RL Train

In [14]:
!./clean_visdata.sh

rm: 无法删除'./data/vis_data/Objects/*': 没有那个文件或目录
rm: 无法删除'./data/vis_data/Trajectory/*': 没有那个文件或目录
rm: 无法删除'./data/vis_data/Fluid/*': 没有那个文件或目录


In [15]:
timesteps = int(1e6)
step = int(1e3)
length =int(timesteps/step) 
print(timesteps,step,length)
last_best_reward = -1e5
cur_steps = 0

1000000 1000 1000


In [16]:
model = SAC("MlpPolicy", env,
             tensorboard_log=network_folder+algofoler, verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
for i in range(0,length):
    model.learn(total_timesteps=step, log_interval=1, reset_num_timesteps=False)
    cur_steps = cur_steps+step
    img_name =algo+ "_steps_{0}.png".format(cur_steps)
    model.save(network_folder+algofoler+'models/'+"_steps_{0}".format(cur_steps))
    rr = evaluate_env(env_test,model,title=img_name,traj_fig_name=network_folder+algofoler+'imgs/trajs/'+img_name,reward_fig_name=network_folder+algofoler+'imgs/rewards/'+img_name)
    if rr>last_best_reward:
        last_best_reward=rr
        model.save(network_folder+algofoler+'models/best')

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22       |
|    ep_rew_mean     | -42.8    |
| time/              |          |
|    episodes        | 1        |
|    fps             | 0        |
|    time_elapsed    | 25       |
|    total timesteps | 22       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 14       |
|    ep_rew_mean     | -26.6    |
| time/              |          |
|    episodes        | 2        |
|    fps             | 0        |
|    time_elapsed    | 33       |
|    total timesteps | 28       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 16.7     |
|    ep_rew_mean     | -31.2    |
| time/              |          |
|    episodes        | 3        |
|    fps             | 0        |
|    time_elapsed    | 57       |
|    total timesteps | 50       |
--------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 15.6     |
|    ep_rew_mean     | -31.2    |
| time/              |          |
|    episodes        | 18       |
|    fps             | 0        |
|    time_elapsed    | 313      |
|    total timesteps | 281      |
| train/             |          |
|    actor_loss      | -4.85    |
|    critic_loss     | 0.556    |
|    ent_coef        | 0.948    |
|    ent_coef_loss   | -0.45    |
|    learning_rate   | 0.0003   |
|    n_updates       | 180      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 15.5     |
|    ep_rew_mean     | -31      |
| time/              |          |
|    episodes        | 19       |
|    fps             | 0        |
|    time_elapsed    | 328      |
|    total timesteps | 294      |
| train/             |          |
|    actor_loss      | -4.86    |
|    critic_loss     | 0.661    |
|    ent_coef 